### 6.4.1. 多输入通道

In [1]:
import  torch
from d2l import torch as d2l

def corr2d_multi_in(X,K):
    # 先遍历"X"和"K"的第0个维度 再把他们加在一起
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))
#zip 将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，
#然后返回由这些元组组成的列表

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [3]:
X.shape

torch.Size([2, 3, 3])

In [4]:
K.shape

torch.Size([2, 2, 2])

In [8]:
# 按照维度0 返回tensor
for x,k in zip(X,K):
    print(x,k)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]]) tensor([[0., 1.],
        [2., 3.]])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]]) tensor([[1., 2.],
        [3., 4.]])


In [9]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
#官方解释：torch.stack沿着一个新维度对输入张量序列进行连接。 序列中所有的张量都应该为相同形状

In [10]:
# 通过将核张量K与K+1（K中每个元素加）和K+2连接起来，构造了一个具有个输出通道的卷积核。
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [18]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [17]:
print(K.shape)
for k in K:
    print(k)
    print(k.shape)

torch.Size([3, 2, 2, 2])
tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])
torch.Size([2, 2, 2])
tensor([[[1., 2.],
         [3., 4.]],

        [[2., 3.],
         [4., 5.]]])
torch.Size([2, 2, 2])
tensor([[[2., 3.],
         [4., 5.]],

        [[3., 4.],
         [5., 6.]]])
torch.Size([2, 2, 2])


### 6.4.3.  1 * 1卷积层

In [21]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape   
    c_o = K.shape[0]         # 卷积核层数  c_o * c_i * 1 * 1
    X = X.reshape((c_i, h * w)) # 将维度1 和 维度2合并
    K = K.reshape((c_o, c_i))   # 只保留前面两维度
    print(X)
    print(K)
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [25]:
X = torch.normal(0, 1, (3, 3, 3)) # 3 * 9 
K = torch.normal(0, 1, (2, 3, 1, 1))  # 2 * 3
Y1 = corr2d_multi_in_out_1x1(X, K)   #2 * 9 -> 2*3*3
Y1

tensor([[-0.5626, -0.2498,  0.4317, -1.1678, -0.0855,  0.6968, -1.0925, -1.0602,
         -0.3155],
        [ 0.6989,  0.3334,  1.5342,  0.7217,  1.9935, -0.4087, -2.2295,  0.1182,
         -1.1846],
        [-1.1232,  1.0720, -0.2503,  1.2869, -0.5216,  0.8511,  0.2139, -1.3661,
          1.3554]])
tensor([[-0.9654, -0.2772, -1.1732],
        [ 1.0578,  0.9481,  1.6589]])


tensor([[[ 1.6672, -1.1089, -0.5483],
         [-0.5825,  0.1419, -1.5579],
         [ 1.4217,  2.5935, -0.9572]],

        [[-1.7959,  1.8302,  1.4960],
         [ 1.5838,  0.9343,  1.7615],
         [-2.9146, -3.2757,  0.7916]]])

### 6.4.4. 小结
* 多输入多输出通道可以用来扩展卷积层的模型。

* 当以每像素为基础应用时，卷积层相当于全连接层。

* 卷积层通常用于调整网络层的通道数量和控制模型复杂性。

2 